In [2]:
import pandas as pd
import re

# === 1. Load Running Order Data ===
df_running_order = pd.read_excel('Running Order Table Pocono Cup 2025.xlsx', sheet_name='MotorSports')
df_running_order = df_running_order.drop(index=0).reset_index(drop=True)
df_running_order.rename(columns={'Unnamed: 0': 'Lap'}, inplace=True)

# === 2. Define caution periods and restart laps ===
caution_periods = [(0, 7), (32, 35), (43, 48), (56, 60), (78, 80), (84, 87), (97, 100), (126, 130)]
restart_laps = [end + 1 for _, end in caution_periods]
restart_laps = [lap for lap in restart_laps if lap != 8]
if 1 not in restart_laps:
    restart_laps.append(1)
restart_laps = sorted(restart_laps)

# === 3. Build choose lookup for (lap, car) ===
choose_file = 'Pocono Choose Line Export - Reorganized.xlsx'
all_sheets = pd.ExcelFile(choose_file).sheet_names

choose_lookup = {}
for sheet in all_sheets:
    match = re.match(r"Car (\d+)", sheet)
    if not match:
        continue
    car_number = int(match.group(1))
    df = pd.read_excel(choose_file, sheet_name=sheet)
    for _, row in df.iterrows():
        lap = row['Lap Number']
        choose_ft = row['Restart Choose (ft)']
        choose_lookup[(lap, car_number)] = choose_ft

# === 4. Process Results ===
results = []

for lap in restart_laps:
    # Special logic for lap 1 (no lap 0)
    if lap == 1:
        running_order_row = df_running_order[df_running_order['Lap'] == 1].iloc[0]
        running_order = running_order_row[list(range(1, 41))].dropna().astype(int).tolist()
        top_16_drivers = running_order[:16]
    else:
        prev_lap = lap - 1
        if prev_lap not in df_running_order['Lap'].values or lap not in df_running_order['Lap'].values:
            continue

        running_order_row = df_running_order[df_running_order['Lap'] == prev_lap].iloc[0]
        running_order = running_order_row[list(range(1, 41))].dropna().astype(int).tolist()

        restart_row = df_running_order[df_running_order['Lap'] == lap].iloc[0]
        top_16_drivers = restart_row[list(range(1, 41))].dropna().astype(int).head(16).tolist()

    for driver in top_16_drivers:
        position_before = running_order.index(driver) + 1 if driver in running_order else None
        choose_distance_ft = choose_lookup.get((lap, driver), None)
        results.append({
            'Lap': lap,
            'Driver': driver,
            'Position_Before_Choose': position_before,
            'Choose Distance (ft)': choose_distance_ft
        })

results_df = pd.DataFrame(results)

# === 5. Compute Lane Choice on original (non-rounded) value ===
def get_lane_choice(choose_ft):
    try:
        if pd.isnull(choose_ft):
            return None
        if 0.000 <= choose_ft <= 14.000:
            return 'Outside'
        elif 14.001 <= choose_ft:
            return 'Inside'
        else:
            return None
    except Exception:
        return None

results_df['Lane Choice'] = results_df['Choose Distance (ft)'].apply(get_lane_choice)

# === 6. Save to Excel ===
results_df.to_excel('results_with_choose_distance_and_lane.xlsx', index=False)
print('Results saved as results_with_choose_distance_and_lane.xlsx')

Results saved as results_with_choose_distance_and_lane.xlsx
